In [1]:
import json
import glob
from GPTUtils import query, prompts
from openai import OpenAI
def save_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
# openai (temp)
openai_api_key = open("openai_api_key").read()
openai_client=OpenAI(api_key=openai_api_key, timeout=30)

In [3]:
# def identify_var_types(all_chunks, openai_client, system_prompt_blocks, user_prompt_blocks, prompt_variables):
#     identify_var_type_prompt_list = []
#     response_format, extract_response_func = None, None
#     for chunk in all_chunks:
#         conversation = chunk['conversation']
#         prompt_variables['conversation'] = conversation_to_string(conversation)
#         identify_var_type_prompt, response_format, extract_response_func = prompts.identify_var_type_prompt_factory(system_prompt_blocks, user_prompt_blocks, prompt_variables)
#         identify_var_type_prompt_list.append(identify_var_type_prompt)
#     identified_var_types = query.multithread_prompts(openai_client, identify_var_type_prompt_list, response_format=response_format, temperature=0.0)
#     if response_format == 'json':
#         identified_var_types = [extract_response_func(i) for i in identified_var_types]
#     for (chunk_index, var_types) in enumerate(identified_var_types):
#         chunk = all_chunks[chunk_index]
#         chunk['var_types'] = var_types
#     return all_chunks

In [4]:
# var_type_definitions = {
#     "driver": "fundamental human causes that lead to certain impacts on the environment to meet basic human needs.",
#     "pressure": "negative phenomena or activities affecting the environment or ecosystems, which are caused by drivers or occur naturally.",
#     "state": "the quantity and quality of physical, chemical, and biological phenomena within a specific timeframe and area.",
#     "impact": "adverse changes in environmental conditions, ecosystem functions, or human well-being.",
#     "response": "any behavior, action, or effort to protect the environment, address environmental issues, or be environmentally friendly."
# }
# for variable_definition_file in glob.glob("contexts/variable_definitions/*.json"):
#     variable_definitions = json.load(open(variable_definition_file))

In [5]:
# prompt_variables = {
#     "var_types": "\n".join([f"{var_type}: {var_type_def}" for var_type, var_type_def in var_type_definitions.items()]),
# }
# prompts = json.load(open('prompts/identify_var_types.json'))
# system_prompt_blocks = prompts['system_prompt_blocks']
# user_prompt_blocks = prompts['user_prompt_blocks']
# all_chunks = []
# for chunk_file in glob.glob("data/v2/tmp/chunk/chunk_summaries_w_ktte/*.json"):
#     chunks = json.load(open(chunk_file))
#     all_chunks += chunks
# # identify the variables in three prompts:
# # First, identify if there are any mentions about the var types by extracting the sentences that mention the var types
# # all_chunks = all_chunks[:1] # test run
# system_prompt_blocks = [prompt_block[1] for prompt_block in system_prompt_blocks]
# user_prompt_blocks = [prompt_block[1] for prompt_block in user_prompt_blocks]
# all_chunks = identify_var_types(all_chunks, openai_client, system_prompt_blocks, user_prompt_blocks, prompt_variables)

# If so, identify the corresponding variables by assigning variables to the sentences
# If no existing variables can be matched, assign it to 'others'

100%|██████████| 598/598 [00:09<00:00, 62.77it/s] 


In [9]:
all_chunks = []
for chunk_file in glob.glob("data/v2/tmp/chunk/chunk_summaries_w_ktte/*.json"):
    chunks = json.load(open(chunk_file))
    all_chunks += chunks
len(all_chunks)
save_json(all_chunks, "data/v2/tmp/pipeline/init/chunks.json")

In [2]:
var_type_definitions = json.load(open('GPTUtils/contexts/var_type_definitions.json'))
prompts = json.load(open('GPTUtils/prompts/identify_var_types.json'))
system_prompt_blocks = prompts['system_prompt_blocks']
user_prompt_blocks = prompts['user_prompt_blocks']
prompt_variables = {
    "var_types": "\n".join([f"{var_type}: {var_type_def}" for var_type, var_type_def in var_type_definitions.items()]),
}
all_chunks = json.load(open("data/v2/tmp/pipeline/init/chunks.json"))
print(len(all_chunks))
# all_chunks = all_chunks[:10]
system_prompt_blocks = [prompt_block[1] for prompt_block in system_prompt_blocks]
user_prompt_blocks = [prompt_block[1] for prompt_block in user_prompt_blocks]
all_chunks = query.identify_var_types(all_chunks, openai_client, system_prompt_blocks, user_prompt_blocks, prompt_variables)
save_json(all_chunks, "data/v2/tmp/pipeline/identify_var_types/chunk_w_var_types.json")

598


100%|██████████| 598/598 [00:20<00:00, 28.84it/s]


In [4]:
all_chunks = json.load(open("data/v2/tmp/pipeline/identify_var_types/chunk_w_var_types.json"))

for chunk in all_chunks:
    var_type_result = chunk['identify_var_types_result']
    var_type_result = list(filter(lambda x: x["var_type"] != 'none', var_type_result))
    chunk['identify_var_types_result'] = var_type_result

save_json(all_chunks, "data/v2/tmp/pipeline/identify_var_types/chunk_w_var_types_filtered.json")


In [3]:
var_type_definitions = json.load(open('GPTUtils/contexts/var_type_definitions.json'))
var_definitions = {}
for var_type in var_type_definitions.keys():
    var_definitions_by_type = json.load(open(f"GPTUtils/contexts/variable_definitions/{var_type}_variables_def.json"))
    var_definitions[var_type] = var_definitions_by_type

prompts = json.load(open('GPTUtils/prompts/identify_vars.json'))
system_prompt_blocks = prompts['system_prompt_blocks']
user_prompt_blocks = prompts['user_prompt_blocks']
prompt_variables = {}
for var_type, var_type_def in var_type_definitions.items():
    prompt_variables[var_type] = {
        "definition": var_type_def,
        "vars": "\n".join([f"{var_name}: {var_def}" for var_name, var_def in var_definitions[var_type].items()])
    }
all_chunks = json.load(open("data/v2/tmp/pipeline/identify_var_types/chunk_w_var_types.json"))
# all_chunks = all_chunks[:10]
system_prompt_blocks = [prompt_block[1] for prompt_block in system_prompt_blocks]
user_prompt_blocks = [prompt_block[1] for prompt_block in user_prompt_blocks]
all_chunks = query.identify_vars(all_chunks, openai_client, system_prompt_blocks, user_prompt_blocks, prompt_variables)
save_json(all_chunks, "data/v2/tmp/pipeline/identify_vars/chunk_w_vars.json")

100%|██████████| 774/774 [00:36<00:00, 20.99it/s]


In [ ]:
import glob
import json
for var_definition_file in glob.glob("GPTUtils/contexts/variable_definitions/*.json"):
    var_definitions = json.load(open(var_definition_file))
    var_def_list = [
        {
            "var_name": var_name,
            "definition": var_def['definition'],
            "factor_type": var_def['factor_type']
        }
        for var_name, var_def in var_definitions.items()
    ]
    with open(var_definition_file, 'w') as f:
        json.dump(var_definitions_by_type, f, indent=4, ensure_ascii=False)